___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://http2.mlstatic.com/D_NQ_NP_2X_960089-MLM26807621582_022018-F.webp" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2025
- <Strong> Email: </Strong>  <font color="blue"> `cd682324@iteso.mx` </font>
___


### <font color= #2E9AFE> Tema: Selección de Variables</font>

La selección de variables es un proceso donde automáticamente se seleccionan aquellos atributos en nuestros datos que contribuyen más a la variable a predecir. 

Las variables irrelevantes o parcialmente relevantes pueden afectar negativamente el rendimiento del modelo.

Beneficios:
- Reducir sobreajuste: menos datos irrelevantes significan menos oportunidades de tomar decisiones basadas en ruido = mejor performance. 
- Modelo más fácil de entender
- Reduce el tiempo de entrenamiento: menos datos significa que el modelo se entrena más rápido


**Tipos de algoritmos de selección de variables**

    - **Métodos de envoltura (wrapper)**: se considera como un problema de búsqueda la selección de un conjunto de variables donde diferentes combinaciones se preparan, evalúan y comparan con otras combinaciones. Se utiliza un modelo predictivo para evaluar una combinación de características y asignar un score basado en la precisión del modelo. 
        - Ejemplo: RFE

    - **Métodos de filtrado**: estos métodos aplican una medida estadística para asignar una puntuación a cada característica. Las características se clasifican según la puntuación y se seleccionan para conservarlas o eliminarlas del conjunto de datos. Los métodos suelen ser univariados y consideran la característica de forma independiente o con respecto a la variable dependiente.
        - Ejemplo: prueba de chi cuadrada, L-Anova, método de correlación, criterio de la varianza

    - **Métodos embebidos (intrinsecos)**: mientras se va creando el modelo el método aprende qué características contribuyen mejor a la precisión. El método más común es el de regularización. 
        - Ejemplo: LASSO, Elastic Net, Ridge, Trees


<img src="https://machinelearningmastery.com/wp-content/uploads/2019/11/Overview-of-Feature-Selection-Techniques3.png" width="550" height="480" align="center"/>


**Mencionando algunas técnicas más comunes:**

- Porcentaje de valores nulos
- Cantidad de variación
- Correlación por parejas
- Mulicolinealidad
- PCA
- Correlación con la variable a predecir (target)
- Forward Selection
- Backward Selection
- Stepwise Selection
- LASSO
- Selección basada en árboles

**¿Cómo elegir las mejores variables?**
No es una respuesta fácil, hay que tratar de varias formas.

# Datos: Cáncer de mama
Los datos se pueden encontrar en:
https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

Se busca clasificar qué si un tumor es maligno (1) o benigno (0). 

Se tiene información de 30 variables


In [ ]:
#Importar librerías
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer

In [ ]:
#cargar datos
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')
df = pd.concat([X,y], axis=1)

In [ ]:
#vistazo datos
df.head()

In [ ]:
#estadistica de los datos

In [ ]:
# observamos la variable de salida
sns.set(style="darkgrid")
ax = sns.countplot(x = y)

**¿Qué tipos de datos tenemos aquí?** 

Las variables de entrada (independientes) son: 

La variable de salida (dependiente) es: 


# <font color= #2E9AFE> Paso 1</font>

**Comienzamos por comprender el negocio**

- Hablar con los expertos en la materia

- Comprende qué características deberían ser importantes según el problema

- Elimina las variables obvias sin sentido (p. ej., identificadores, variables sin valor)


# <font color= #2E9AFE> Paso 2</font>

**Limpieza previa a los datos (solo si es necesario)**


In [ ]:
#Escalamiento --> SOLO A LAS VARIABLES DE ENTRADA
x_scaled = X
x_scaled =  (x_scaled - x_scaled.mean())/(x_scaled.std())
df_scaled = pd.concat([x_scaled,y], axis=1)
df_scaled.head()

# <font color= #2E9AFE> Paso 3 Visualización y Cuestionamiento</font>

La visualizacion nos ayuda a darnos una idea de qué variables podrían ayudar o no al modelo

In [ ]:
def violin_plot(beginning,end):
    data = pd.concat([y,x_scaled.iloc[:,beginning:end]],axis=1)
    data = pd.melt(data, id_vars="target", var_name="features", value_name='value')
    plt.figure(figsize=(10,10))
    sns.violinplot(x="features", y="value", hue="target", data=data, split=True, inner="quart")
    plt.xticks(rotation=45)

violin_plot(0,10)

mean radius, mean perimeter, mean area, compactness mean, mean concavity, mean concave points están bien separados entre tumores malignos y benignos. Estas 6 características serían buenas candidatas para el modelo.

Por el contrario, mean fractal dimension tiene la misma mediana para ambos tipos de tumores, por lo que no sería un buen candidato para el modelo.

In [ ]:
violin_plot(20,31)

En el gráfico anterior, notamos algunas similitudes entre el wort radius y el worst perimeter. Si dos violines se parecen, podría indicar una correlación entre las características, y si dos características están correlacionadas, uno puede preguntarse si es posible (o no) eliminar una.

In [ ]:
sns.regplot(data = x_scaled, x = "worst radius", y="worst perimeter", color="#ce1414")

In [ ]:
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(X.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

 Con este análisis me atrevería a sugerir remover las siguientes variables:
 
 - mean radius
 - mean perimeter
   
 - mean compactness
 - mean concave points

- radius error
- perimeter error

- worst radius
- worst perimeter

- compactness error
- concave points error

- mean compactness
- mean concave points

- worst textrue

- worst area

**NOTA:** no lo voy a hacer por lo pronto, porque podría remover alguna variable que es estadísticamente significativa para el target (tumor maligno o benigno), así que primero aplicaré un método de filtrado y luego removeré las variables que están muy correlacionadas

# <font color= #2E9AFE> Métodos de Filtrado (si contamos con muchas variables)</font>

Si estamos comenzando con la evaluación del dataset. Podríamos iniciar con métodos de filtrado para tomar en cuenta qué variables podríamos remover ya que tenemos muchas variables en nuestros datos.


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# ANOVA
selector = SelectKBest(score_func=f_classif, k='all')
f_vals, p_vals = f_classif(x_scaled, y)
significant = np.where(p_vals < 0.05)[0]
print("Variables significantes:", X.columns[significant].tolist())

In [ ]:
# seleccionamos aquellas variables que son significativas
df_reduced = df_scaled[['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'radius error', 'perimeter error', 'area error', 'compactness error', 'concavity error', 'concave points error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension','target']]

In [ ]:
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(df_reduced.drop(columns='target').corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

**Opcional**

Observo que quedaron todavía variables muy correlacionadas, por lo tanto ahora sí quitaré las variables correlacionadas que vimos anteriormente

In [ ]:
drop_list = ['mean perimeter','mean radius','mean compactness','mean concave points','radius error','perimeter error',
             'worst radius','worst perimeter','worst compactness','worst concave points',
             'compactness error','concave points error','worst texture','worst area']
df_reduced = df_reduced.drop(drop_list,axis = 1 )
df_reduced.shape

El siguiente paso sería aplicar una selección de variables dirigida al modelo. 

Podemos usar

- Wrapper methods: si tenemos modeos como regresión logística, regresión lineal o máquinas de vector soporte
- Métodos embebidos: si vamos a utilizar algún modelo en específico que ya tenga dentro del modelo un método de selección de variables


# <font color= #2E9AFE> Métodos de Envoltura (Wrapper)</font>

### Eliminación Recursiva de Características (RFE)

Funciona eliminando atributos de forma recursiva y construyendo un modelo sobre los atributos que quedan.

Usa la precisión del modelo para identificar qué atributos (y combinación de atributos) contribuyen más a predecir el objetivo.

Vamos a utilizar la regresión logística para seleccionar las características principales. 

**Por lo general se usa dentro del cross-validation o en pipelines para evitar el overfitting** 

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

selector = RFECV(LogisticRegression(max_iter = 10000), cv=5)

X = df_reduced.drop(columns="target")
y = df_reduced['target']
selector.fit(X,y)
X_selected = X.loc[:, selector.support_]

In [ ]:
X_selected

# <font color= #2E9AFE> Métodos embebidos (intrínsecos)</font>

Existen algunos modelos (vamos a irlos viendo) que dentro de su construcción ya hacen una selección de variables. Uno de ellos son los modelos basados en árboles como el "Random Forest". 

Construimos un Clasificador de "Random Forest"

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

#creamos el modelo del random forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

# Vamos a quedarnos con las variables que tengan una importancia arriba del promedio
sfm = SelectFromModel(rf, threshold='median')
sfm.fit(X, y)
X_embedded = X.loc[:, sfm.get_support()]
print("Variables seleccionadas metodo embebido dentro del random forest:", X_embedded.columns.tolist())

In [ ]:
#Visualizamos la importancia de las variables
import matplotlib.pyplot as plt

importances = rf.feature_importances_
feature_names = X.columns

sorted_idx = importances.argsort()[::-1]
plt.figure(figsize=(12, 6))
plt.barh(feature_names[sorted_idx], importances[sorted_idx])
plt.title("Feature Importances (Random Forest)")
plt.xlabel("Importance Score")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


## Comparando el performance de los modelos utilizando los distintos métodos

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

def evaluate_model(X_subset, y, model):
    scores = cross_val_score(model, X_subset, y, cv=5, scoring='accuracy')
    return scores.mean()

print("Método de filtrado:", evaluate_model(df_reduced.drop(columns="target"), y, LogisticRegression(max_iter=10000)))
print("Método Wrapper (RFECV):", evaluate_model(X_selected, y, LogisticRegression(max_iter=10000)))
print("Método embebido (Random Forest):", evaluate_model(X_embedded, y, RandomForestClassifier(random_state=42)))


In [ ]:
# cuantas variables eligió cada método
print("Metodo filtrado:", df_reduced.drop(columns="target").shape[1])
print("Metodo wrapper:", X_selected.shape[1])
print("Metodo embebido:", X_embedded.shape[1])


### ¿Cuál es el mejor método?

...no hay...

Se tiene que hacer experimentación para ver qué método funciona mejor para el problema en específico. 

## Conclusion

La cantidad ideal de variables NO siempre es la menor: hay que observar el rendimiento del modelo, la interpretabilidad y las necesidades del negocio.

### Referencias:

- https://www.stratascratch.com/blog/feature-selection-techniques-in-machine-learning/
- https://scikit-learn.org/stable/auto_examples/feature_selection/plot_f_test_vs_mi.html#sphx-glr-auto-examples-feature-selection-plot-f-test-vs-mi-py
- https://scikit-learn.org/stable/modules/feature_selection.html


<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Sara E. Rodríguez.
</footer>